# Flow Explanation

In this notebooks we'll explain the entire flow of building the system and using it.

## Step 1 - Creating the dataset using the Dataset Builder

Our first task, in [data_gathering](https://github.com/TAU-DB/guided-ds/tree/master/data_gathering) is to create a repository of reproducible data science code-snippets. 
We'de like to methodically download datasets, jupyter notebooks, and all relevant metadata from Kaggle.com (online community of data scientists). <br>

Then, we'de like to parse this repository into tsv files that will be used as input to our models in the following stages. <br>
This process is demonstated in the [example.ipynb](https://github.com/TAU-DB/guided-ds/blob/master/data_gathering/example.ipynb) notebook.

As explained in the notebook, Paths and other constants can be configuered in [data_gathering/consts.py](https://github.com/TAU-DB/guided-ds/blob/master/data_gathering/consts.py)

If we followed the steps in the example notebook <u>up to step3</u>, we downloaded the wanted datasets, jupyter notebooks, and all relevant metadata, and parsed the repository into a tsv file called ```cells.tsv``` that contains for each cell: a unique cell id, relevant notebook, username (of author), cell's source code.

<em> **Note** - file paths can be configured in the [consts.py](https://github.com/TAU-DB/guided-ds/blob/master/data_gathering/consts.py) file. The file for the next step is stored in ```consts.CELLS_TSV```</em>

<em> **Note2** - our existing cells.tsv file is stored in the [Data](https://github.com/TAU-DB/guided-ds/tree/master/Data) folder. </em>

## Step 2 - Classifying the cells using the Workflow Stage Classifier


Our next task, in [Classification](https://github.com/TAU-DB/guided-ds/tree/master/Classification) is to classify the cells to the relevant Data Science workflow stage (Imports/Load Data/Data Exploration/Data Preparation and Cleaning/Model Training and Parameter Tuning/Model Evaluation).

### To recreate:

Use the [Classification/example_classifier.ipynb](https://github.com/TAU-DB/guided-ds/blob/master/Classification/example_classifier.ipynb) notebook.<br>

In the notebook we used ```consts.CELLS_TSV``` (the file we created in Step1) as input, when the cells are still unlabeled (no info about their workflow stage). 
Using [snorkel](https://hazyresearch.github.io/snorkel/), with our labeling functions defined in [Classification/utils/LF_utils.py](https://github.com/TAU-DB/guided-ds/blob/master/Classification/utils/LF_utils.py) and the hand-labeled cells that are stored in a path configured by ```consts.GOLD_LABELS```, we tag the data to be able to use supervised-learning methods.

Our snorkel labeled datais saved in the folder ```consts.DATA_FOLDER``` as ```input.tsv```.

Then, using the snorkel tagged data we train a supervised LSTM model That will classify the cell to the relevant data science workflow stage. 

The trained classifier (LSTM) is saved in the folder defined by ```consts.CLASSIFIER``` in the ```tokenizer.pickle```, ```model.json``` and ```model.h5``` files.

Then, Using the trained model we tag all of the cells in ```consts.CELLS_TSV``` (inplace, unless you change the path of ```consts.TAGGED_CELLS_TSV```).


### Our Process:

In the [Exploration_and_WeakSupervision.ipynb](https://github.com/TAU-DB/guided-ds/blob/master/Classification/Exploration_and_WeakSupervision.ipynb) we explore our data and use snorkel to tag it. 
The input file we used for this notebook is found in the [Classification/input](https://github.com/TAU-DB/guided-ds/tree/master/Classification/input).
Our hand-labeled cells are stored in [Classification/input/gold_labels.tsv](https://github.com/TAU-DB/guided-ds/blob/master/Classification/input/gold_labels.tsv).

In the [Classification.ipynb](https://github.com/TAU-DB/guided-ds/blob/master/Classification/Classification.ipynb) notebook we train the supervised LSTM model.

Our Trained classifier is stored in the classification folder in the ```tokenizer.pickle```, ```model.json``` and ```model.h5``` files.

Our tagged data is stored in [Data/tagged_cells.tsv](https://github.com/TAU-DB/guided-ds/blob/master/Data/tagged_cells.rar) (it is compressed).



## Step 3 - Creating the input files for the Chatbot Training

In order to train our chatbot model, that will eventually generate next-line recommendations, we need a file containing input and ouput pairs.<br>
In [data_gathering/example.ipynb](https://github.com/TAU-DB/guided-ds/blob/master/data_gathering/example.ipynb) Step5 we show how to generate those files.

Specifically for our recommendation engine we used 3lines-nextline pairs of each input stage seperately. 
Those files were generated as appears in the notebook, with ```cells_to_masked_lines()```.

The generated pairs files are saved to the path configured by: ```consts.EXPLORE_TSV```, 
```consts.PREP_TSV```, ```consts.LOAD_TSV```, ```consts.TRAIN_TSV```, ```consts.EVAL_TSV```, ```consts.IMPORT_TSV```.

The final files that we used to train our models are- Import.tsv, Load.tsv, Explore.tsv, Prep.tsv, Train.tsv, Eval.tsv. They are stored in the [Data](https://github.com/TAU-DB/guided-ds/tree/master/Data) folder.

## Step 4 - Training the chatbot Model(s)

Now that we have the necessary input-output pairs file, we use it (by setting ```datafile``` accordingly) to train a chatbot model in [Chatbot/example_chatbot.ipynb](https://github.com/TAU-DB/guided-ds/blob/master/Chatbot/example_chatbot.ipynb).

All of the chatbot parameters are tunable inside the notebook.<br>

The model is automatically saved during training, every ```save_every``` iterations, in ```consts.CHATBOT_MODELS_FOLDER```.

**Note**: you need to run the process multiple time, once for each pairs file (for each stage of th input cell).
after training rename the wanted model from ```consts.CHATBOT_MODELS_FOLDER```, store it where you'd like.


You can see our training process, with a little more details and evaluation, in [Chatbot/Jupyter_Cells_Chatbot_Model.ipynb](https://github.com/TAU-DB/guided-ds/blob/master/Chatbot/Jupyter_Cells_Chatbot_Model.ipynb) (again, we ran this process multiple times for a lot of different inputs).

Our trained models are stored in [Chatbot/Models](https://github.com/TAU-DB/guided-ds/tree/master/Chatbot/Models).

## Step 5 - Use the trained models in the Recommendation Engine

After you trained your models for each stage, configure the paths to the trained models in: ```consts.EXPLORE_MODEL```, ```consts.PREP_MODEL```, ```consts.LOAD_MODEL```, ```consts.TRAIN_MODEL```, ```consts.EVAL_MODEL```, ```consts.IMPORT_MODEL```.

In [Chatbot/example_engine.ipynb](https://github.com/TAU-DB/guided-ds/blob/master/Chatbot/example_engine.ipynb) you can use yout trained classifier and models as a complete recommendation system.

The flow of the recommendation process is explained in the readme file of the [Recommendation Engine](https://github.com/TAU-DB/guided-ds/tree/master/Chatbot).

To check our final recommendation system, see- [Chatbot/Recommendation_Engine.ipynb](https://github.com/TAU-DB/guided-ds/blob/master/Chatbot/Recommendation_Engine.ipynb)


<em> **Note** - The Recommendation_Engine file is the system itself. It is usable without running all other steps. The other steps are required to create such a system of your own or modify the system. </em>